# Les migrations

## Introduction

Les **migrations** constituent le moyen désormais convenu de créer et modifier les schémas de bases de données relationnelles selon les pratiques des gestions de versions.

Elles permettent de garder une trace des modifications successives (dans la base de données elle-même) et donc de pouvoir revenir à un état antérieur du schéma en cas de besoin.

Dans le cas de `Laravel`, les migrations constituent un point essentiel de la définition du modèle de données puisque les entités (appelées de manière assez confuse « _modèles_ », contennent très peu de choses.

Les migrations ne sont donc pas ici, comme dans `Symfony`, une simple opération automatique transcrivant la structure d'une entité en son équivalent SQL, mais des classes dans lesquelles le développeur devra manuellement coder les modifications successives du schéma de la base de données de sbn application.

## Pré-requis

Le rôle d'une migration est de modifier le schéma d'une base de données.Il faut donc que celle-ci existe au préalable.

Vous devrez donc vérifier que :

1. le fichier `.env` contient les bonnes informations de configuration de la connexion, par exemple :

In [ ]:
DB_CONNECTION=mysql
DB_HOST=127.0.0.1
DB_PORT=8889
DB_DATABASE=dm22_amap
DB_USERNAME=root
DB_PASSWORD=root

2. le fichier de configuration `config/database.php` importe ces données (c'est normaleent le cas par défaut)
3. vous avez créé la base de données sur votre serveur préféré.

> **Note** 
>
> Il n'existe pas de commande pour créer la base de données, aussi étrange que cela puisse paraître. Nous verrons plus tard que créer cette commande est pourtant très simple.

## Une migration simple

Pour créer une migration, il existe une commande :

In [ ]:
%%bash

php artisan make:migration nouvelle_migration

où `nouvelle_migration` est un nom arbitraire.

Cette commande crée un classe dans le dossier `/database/migrations` et celle-ci contient principalement deux méthodes : `up` et `down`.

- `up` applique la migration
- `down` permet de revenir à un état antérieur du schéma de la base.

Le nom de la migration est un indicateur important pour `Laravel`, qui essaie de deviner quele est son propos. Un nom comme `create_product_table` indiquera à `Laravel` qu'il doit incorporer une fonction de création de table.

### Relation entre modèle et migration

D'une manière générale, une première migration peut être créée lorsqu'un nouveau modèle est ajouté à l'application, ne serait-ce que pour créer la table correspondante. Cela se traduit par la commande :

In [ ]:
%%bash

php artisan make:model Product --migration

> **Note** L'option `--migration` est contenue dans l'option globale `--all`

Toutefois, vous pouvez aussi préférer configurer le modèle (cf. [Modèles]()), puis créer la migration afin de bénéficier de ces éléments de configuration (personnalisation de la clef primaire, gestion des dates administratives, etc.)

### Création d'une table

Pour créer une table via une migration, il existe une méthode statique `create`.

In [ ]:
%%php

public function up()
{
    Schema::create('contacts', function (Blueprint $table) 
    {
            $table->id();
            $table->string('name');
            $table->string('mobile_no');
            $table->boolean('status');
            $table->timestamps();
    });
}

Le point principal ici est la classe `Blueprint`, qui possède une grande variété de méthodes, sur la base des types SQL, pour créer les colonnes de la table.

Vous trouverez la référence de toutes ces fonctions dans la [documentation de Laravel](https://laravel.com/docs/10.x/migrations#columns), tout comme la liste des [types disponibles](https://laravel.com/docs/10.x/migrations#available-column-types).

### Application des migrations

Une fois une migration définie, il ne reste plus qu'à l'appliquer. il y a pour cela une autre commande :

In [ ]:
%%bash

php artisan migrate

Si vous regardez dans votre base de données, vous constaterez qu'une nouvelle table, appelée par défaut `migrations` a été créée. C'est là que tout l'historique des modifications du schéma de votre base sera conservé.

### Modification d'une table

Une fois une table créée, si vous voulez modifier sa structure, il existe plusieurs méthodes :

In [ ]:
%%php

//  Modification des colonnes
Schema::table('products', function(Blueprint $table) {
    /* ... */
});
        
// Renommage d’une table
// Attention à d’éventuelles clefs étrangères pointant vers cette table !
Schema::rename('types', 'product_types');
        
// Suppression d’une table
Schema::dropIfExists('types');

## Autres fonctionnalités

### Informations par défaut

Lorsque l'on crée une nouvelle migration, Eloquent prévoit déjà des colonnes par défaut qui sont :
- `id`, pour la clef primaire ;
- `created_at` et `updated_at`, comme dates de gestion des données.

#### Clef primaire

Eloquent suppose que la clef primaire s'appelle `id` et qu'elle est à la fois numérique et auto-incrémentée. C'est le sens de la méthode `$table->id()`.

Totefois, nous pouvons définir notre propre clef primaire en définissant une colonne et en la déclarant comme clef primaire :

In [ ]:
%%php

$table->string('email', 128);
$table->primary('email');

Au passage, on remarque les paramètres optionnels pour les méthodes de types. Ici, la longueur d'affichage de `email` est de 128 caractères.

La clef primaire peut éventuellement être définie sur plusieurs colonnes, en passant un tableau. En revanche, il ne semble pas possible de prendre des fragments des colonnes (p.ex. les 30 premiers caractères de `email`).

#### Dates de gestion

La fonction `timestamps()` crée les deux colonnes `created_at`et `updated_at`. Naturellement, ces informations sont tout à fait facultatives. Si votre schéma ne les prévoit pas, vous pouvez alors supprimer la ligne de code.

### Index et clefs étrangères

En dehors de la clef primaire, vous trouverez dans `Blueprint` ce qu'il faut pour déclarer des index divers.

#### Index

Pour les index simples, il suffit de la méthode `index` :

In [ ]:
%%php

$table->string('email', 128);
$table->index('email');

Là non plus, Eloquent ne prévoir pas de créer unindex sur une portion de la chaîne de caractères. En revanche, l'index peut être composé :

In [ ]:
%%php

$table->string('prenom');
$table->string('nom');
$table->index([prenom', 'nom'], 'index_nom_prenom');

Comme on le voit, il est possible de donner un nom explicite à l'index, mais dans les faits cela est rarement utile.

#### Valeurs uniques

De la même manière, si une colonne requiert des valeurs toutes diférentes, vous pourrez utiliser la méhode `unique()` :

In [ ]:
%%php

$table->string('email', 128);
$table->unique('email');

#### Clefs étrangères

Pour définir une clef étrangère, la syntaxe est un tout petit plus longue :

In [ ]:
%%php

$table->string('cusomerNumber', 128);
$table->foreign('customerNumber')->references('id')->on('customers');

où :
- `references` désigne la colonne cible dans la table distante (généralement une clef primaire)
- `on` désigne le nom de la table distante.

Lorsque l'on crée une base de données, l'ordre des migrations peut avoir de l'importance. Rn effet, la clef étrangère ne pourra être créée que si la clef primaire correpondante existe. En cas d'inversion, il est toujours possible de modifier le schéma d'une table avec la méthode `Schema::table` :

In [ ]:
%%php

Schema::create('customers', function (Blueprint $table) { /* ...*/});

Schema::table('payments', function (Blueprint $table) {
    $table->foreign('customerNumber')->references('id')->on('customers');
})

Dans l'exemple ci-dessus, la migration va créer la tables `customers` pui modifier `payments`oury intégrer la clef étrangère.

#### Indexation de texte intégral

Pour certains besoins, il est également possible, pour les colnnes des type texte, d'ajouter un index pour accélerer la recherche :

In [ ]:
%%php

$table->text('description');
$table->fullText('email');

## Modification du schéma

Lors de la première migration méthode `up()` est assez simple, car tout est à créer. 

Par la suite, nous serons peut-être amenés à supprimr ddes colonnes, des index, changer de nom, etc. Cela est typiquement le cas lorsqu'il faudra implémente un méthode `down()` pour restaurer un état antérieur.

Parallèlement à ce que nous avons décrit précédemment, `Blueprint` dispose de méthodes pour toutes ces tâches.

## Commandes pour la gestion des migrations

Nous avons vu pour créer une classe de migration, nous avions recour à la commande :

In [ ]:
%%bash

php artisan make:migration

Puis pour effectuer la migration :

In [ ]:
%%bash

php artisan migrate

Pour revenir à un état antérieur, utilisez la commande :

In [ ]:
%%bash

php artisan migrate:rollback --step=3

En ajoutant l'option `step`, pous pourrez revenir de plusieurs étapes en arrière.

Vous pouvez également annuler une migration et en lancer une nouvelle (p.ex. en cas d'erreur) :

In [ ]:
%%bash 

php artisan migrate:refresh

Et enfin, si vous voulez repartir de zéro :

In [ ]:
%%bash 

php artisan migrate:fresh

Vous pouvez également faire une sauvegarde de l'historique des migrations avec la commande :

In [ ]:
php artisan schema:dump --prune

Dans ce cas `Laravel`créera un fichier dans le répertoire `database/schema` pour synthétiser en une seule migration toutes celles que vous avez déjà exécutées précédemment. L'option `--prune` indique que les migrations passées doivent être écrasées.

Si vous voulez migrer votre base de données, `Laravel` commencera par exécuter le instructions du fichier de « _dump_ » avant d'éventuelles migrations restantes.